# Singularity

References: https://sylabs.io/docs/

# Container technology comparison

<img src="https://tin6150.github.io/psg/fig/vm_vs_container.png">



[Container technology comparison](https://tin6150.github.io/psg/blogger_container_hpc.html)

## Build

### From pre-built images

Build command can only be performed by root or sudoer

Command syntax:  
```bash
$ {sudo} singularity build {image_name} {image_uri or definition file}
```


Examples:
```bash
$ sudo singularity build ubuntu.sif library://ubuntu
$ sudo singularity build lolcow.sif docker://godlovedc/lolcow
```

Sylabs Container Library: https://cloud.sylabs.io/library  
DockerHub: https://hub.docker.com/  


### From definition file

lolcow.def
```bash
BootStrap: library
From: ubuntu:16.04

%post
    apt-get -y update
    apt-get -y install fortune cowsay lolcat

%environment
    export LC_ALL=C
    export PATH=/usr/games:$PATH

%runscript
    fortune | cowsay | lolcat

%labels
    Author GodloveD
```

Build `lolcow.sif` from the `lolcow.def` definition file.
```bash
$ sudo singularity build lolcow.sif lolcow.def
```

### About `--sandbox`

```bash
$ sudo singularity build --sandbox sandbox.simg library://centos:latest
$ ls -ld sandbox.simg/
dr-xr-xr-x. 18 root root 4096 May 19  2019 sandbox.simg/
```
Do you see that its a directory?  
You can also `cd` into it.

```bash
$ cd sandbox.simg/
$ ls
bin  boot  dev  environment  etc  home  lib  lib64  media  mnt  opt  proc  root  run  sbin  singularity  srv  sys  tmp  usr  var
```
Looks just like a regular old linux root directory!

## Interacting with images

### Shell

```bash
$ singularity shell lolcow.sif

Singularity lolcow.sif:~> whoami
username
Singularity lolcow.sif:~> id
uid=1000(username) gid=1000(username) groups=1000(username)
Singularity lolcow.sif:~> exit

$ sudo singularity shell lolcow.sif
Singularity lolcow.sif:~> whoami
root
Singularity lolcow.sif:~> id
uid=0(root) gid=0(root) groups=0(root)
```

Use the `--writable` option to persist changes in the container.  

Readonly Example:
```bash
# Shell inside the container 
$ singularity shell sandbox.simg

# Try and make a file in the root directory
Singularity sandbox.simg:~> echo "Hello World" > /hello.txt
bash: /hello.txt: Read-only file system
```
Error. The file cannot be made.  

Give the `--writable` option to persist changes in the container.  
```bash
# shell back inside the container
# You probably need to shell as root unless you changed the permission of the container
$ sudo singularity shell --writable sandbox.simg

# Make the same file as last time, only it works this time.
Singularity sandbox.simg:~> echo "Hello World!" > /hello.txt

# Exit the container and come back in
Singularity sandbox.simg:~> exit
$ singularity shell sandbox.simg

# check if the previously made file exists
Singularity sandbox.simg:~> cat /hello.txt
Hello World!
```

You can also now install linux packages with the `--writable` option enabled.

```bash
# shell in as root with writable enabled
$ sudo singularity shell --writable sandbox.simg

# Let's install python3
Singularity sandbox.simg:~> yum install -y python3

# Fuck it.. Let's also install the Python package FuckIt
# https://github.com/ajalt/fuckitpy
Singularity sandbox.simg:~> pip3 install fuckit
```

### Executing Commands

`exec` subcommand is used to execute a command within the container.  
For example, the following command instructs the container `lolcow.sif` to execute `cowsay moo`:

```bash
$ singularity exec lolcow.sif cowsay moo
```

### Running a container

```bash
$ singularity run lolcow_latest.sif
```

### Binding (`--bind` / `-B`)

The benefits of using a container is that the container and the host system has a completely distinct environment. So changing the environment in one does not affect the environment of the other. However, separation of the environment also imply a separation of the file system between the two.

As shown so far, the host filesystem has access to `sandbox` container images but not the other way around. But one major use case of containers are to set up a system where the softwares in the container manipulates and changes the files in the host file system. 

So how can we get our containers to access the host file system? The answer: binding.


```bash
$ echo 'Hello from the host!' > ~/hello_host.txt
$ singularity shell sandbox.simg
Singularity sandbox.simg:~> cat ~/hello_host.txt
Hello from the host!
```
By default Singularity bind mounts `/home/$USER`, `/tmp`, and `$PWD` into your container at runtime.  

By using the `--bind` command, we can mount additional host file system directories onto directories inside the container.